<img src="images/borehole_graph.png" ></img>

<br>

# Borehole Graph Data Model Example

In this notebook, we will use <a href="https://steno3d.com">Steno3D</a> to view and share data throughout the steps of a lithological borehole graph from neo4j. 

## Login to Steno3D

The first thing you need to do is login to Steno3D. You can <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key.

In [1]:
import steno3d

In [2]:
steno3d.login()


Oh no! Your version of steno3d is out of date.

Your version: 0.3.11
Current version: 0.3.12

Please update steno3d with `pip install --upgrade steno3d`.



>> Welcome to the Python client library for Steno3D!

Credentials file found: /Users/mmorley/.steno3d_client/credentials
Accessing API developer key for @mmorley
Welcome to Steno3D! You are logged in as @mmorley


In [3]:
prelim_data = steno3d.Project(
    title='Neo4j Borehole Experiment',
    description='An experiment in modeling 3d borehole data with neo4j.',
    public=True
)

## Query the Drill Collar Locations from Neo4j using Neo4j Driver

First step is to use a cypher query to extract the data from Neo4j. We will use the Neo4j Python driver for this, through the jupyter notebook. 

The Steno3d app requires collar data to be in a nested array of form: [[x,y,z]...[x,y,z]], so we require a neo4j query that returns the data through the python driver. 

One thing to note is that Steno3d uses cartesian cordinates, so we will use the original NAD87 based locations instead of the Neo4j spatial coordinates/points, which is why we preserved these. Ahh the joys of spatial data!

match (c:Collar) with [c.E_10TM83,c.N_10TM83,c.Location.z] as location  return collect(location) as location

The results of this statement will be placed into a variable for steno. 

In [4]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def borehole_data(tx):
    records=tx.run("match (c:Collar) where c.Name in ['99','98','97','96','95','88','89','90','91','92','73','72','71','70','69'] "
                         "with [c.E_10TM83,c.N_10TM83,c.Location.z] as location "
                         "return collect(location) as location")
    return records
        
with driver.session() as session:
    results=session.read_transaction(borehole_data).data(0)
    nodes=[]
    i = 0
    for record in results:
        target= i
        i += 1
        for item in record['location']:
            nodes.append(item)
            source= i
            i+=i
    # print(nodes)
    drill_locations=steno3d.Mesh0D(vertices=nodes) 


We only need the locations of these points, no other data, so let's construct our <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">point</a> resource now. We will make the points orange so they stand out when plotted.

In [5]:
drill_points = steno3d.Point(
    project=prelim_data,
    title='Proposed Drill Locations',
    mesh=drill_locations,
    opts=dict(color='orange')
)

After uploading these points to our project on <a href="https://steno3d.com">steno3d.com</a>, let's print the url of our project to share.

In [6]:
drill_points.upload()

Verifying your quota for public projects...
This PUBLIC project will be viewable by everyone.
Total progress: 100% - Uploading: project Neo4j Borehole Experiment
Complete!
https://steno3d.com/resource/point/KFmBEKSaXgNzd9IKD7gL


'https://steno3d.com/resource/point/KFmBEKSaXgNzd9IKD7gL'

In [7]:
print(prelim_data.url)

https://steno3d.com/app/N513R05zK8ZwyqnkBBN9


## Plot the Borehole Data

After discussing the preliminary data project online with collaborators, it was decided to go ahead with borehole drilling. Now the data is back and available to be explored with. Let's make a new project to contain the results of the drilling.

In [23]:
result_data = steno3d.Project(
    title='Neo4j Borehole Experiment',
    description='An experiment in modeling 3d borehole data with neo4j.',
    public=True
)

To display our borehole data, we first need a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">mesh 1D</a> based on their spatial geometry. This is imported from the Neo4j graph database model. The `view_type` option sets the lines mesh to initially display as fixed-width, extruded "tubes" rather than true 1D lines.

// borehole vertex list
match (c:Collar)-[:NEXT_INTERVAL*]-(i:Interval) 
with c,i
with [c.E_10TM83,c.N_10TM83,i.location.z] as location
return collect(location)

// borehole segments 
match (c:Collar)-[:NEXT_INTERVAL*]-(i:Interval) 
with c,i
with [i.FromDepth, i.ToDepth] as location
return collect(location)

In [24]:
def borehole_data(tx):
    records=tx.run("match (c:Collar) where c.Name in ['99','98','97','96','95','88','89','90','91','92','73','72','71','70','69'] "
    "with c  "
    "match (c)-[:NEXT_INTERVAL*]-(i:Interval)  "
    "with c, i order by i.location.z desc  "          
    "with [c.E_10TM83,c.N_10TM83,i.elev] as location return collect(location) as locations")
    return records
        
with driver.session() as session:
    results=session.read_transaction(borehole_data).data(0)
    print(results)
    nodes=[]
    i = 0
    for record in results:
        target= i
        i += 1
        for item in record['locations']:
            nodes.append(item)
            source= i
            i+=i
    borehole_vertices=nodes



[{'locations': [[656585.0, 5939239.0, 687.0], [656533.0, 5940849.0, 686.0], [658216.0, 5939298.0, 682.0], [656533.0, 5940849.0, 679.0], [656585.0, 5939239.0, 677.0], [658216.0, 5939298.0, 675.0], [661419.0, 5941022.0, 675.0], [663105.0, 5939454.0, 673.0], [663071.0, 5941066.0, 673.0], [656480.0, 5942480.0, 673.0], [661365.0, 5942625.0, 672.0], [658106.0, 5942539.0, 672.0], [659789.0, 5940965.0, 672.0], [658166.0, 5940910.0, 672.0], [661475.0, 5939384.0, 672.0], [661419.0, 5941022.0, 672.0], [659865.0, 5939355.0, 671.0], [659734.0, 5942595.0, 670.0], [659865.0, 5939355.0, 670.0], [662997.0, 5942700.0, 670.0], [659734.0, 5942595.0, 669.0], [661475.0, 5939384.0, 669.0], [658106.0, 5942539.0, 667.0], [663071.0, 5941066.0, 667.0], [661419.0, 5941022.0, 667.0], [656480.0, 5942480.0, 667.0], [659789.0, 5940965.0, 666.0], [658166.0, 5940910.0, 666.0], [663105.0, 5939454.0, 665.0], [661475.0, 5939384.0, 665.0], [663105.0, 5939454.0, 664.0], [663071.0, 5941066.0, 664.0], [659865.0, 5939355.0, 66

In [25]:
def borehole_data(tx):
    records=tx.run("match (c:Collar)  where c.Name in ['99','98','97','96','95','88','89','90','91','92','73','72','71','70','69']"
    "with c "
    "match (c)-[:NEXT_INTERVAL*]-(i:Interval) "   
    "with c, i order by i.location.z desc "               
    " with i,[ toInt(i.FromDepth),toInt(i.ToDepth) ] as segment return collect(segment) as locations")
    return records
        
with driver.session() as session:
    results=session.read_transaction(borehole_data).data(0)
    print(results)
    nodes=[]
    i = 0
    for record in results:
        target= i
        i += 1
        for item in record['locations']:
            nodes.append(item)
            source= i
            i+=i
    borehole_segments=nodes
    print(borehole_segments)
    print(borehole_vertices)
borehole_mesh = steno3d.Mesh1D(
    vertices = borehole_vertices,
    segments = borehole_segments,
    opts = dict(
        view_type='boreholes'
    )
)


[{'locations': [[0, 10], [0, 7], [0, 7], [7, 9], [10, 13], [7, 18], [0, 3], [0, 8], [0, 6], [0, 6], [0, 11], [0, 5], [0, 6], [0, 6], [0, 3], [3, 8], [0, 1], [0, 1], [1, 8], [0, 10], [1, 10], [3, 7], [5, 12], [6, 9], [8, 13], [6, 9], [6, 13], [6, 8], [8, 9], [7, 13], [9, 13], [9, 14], [8, 13], [11, 13], [12, 14], [10, 13], [10, 14], [14, 18]]}]
[[0, 10], [0, 7], [0, 7], [7, 9], [10, 13], [7, 18], [0, 3], [0, 8], [0, 6], [0, 6], [0, 11], [0, 5], [0, 6], [0, 6], [0, 3], [3, 8], [0, 1], [0, 1], [1, 8], [0, 10], [1, 10], [3, 7], [5, 12], [6, 9], [8, 13], [6, 9], [6, 13], [6, 8], [8, 9], [7, 13], [9, 13], [9, 14], [8, 13], [11, 13], [12, 14], [10, 13], [10, 14], [14, 18]]
[[656585.0, 5939239.0, 687.0], [656533.0, 5940849.0, 686.0], [658216.0, 5939298.0, 682.0], [656533.0, 5940849.0, 679.0], [656585.0, 5939239.0, 677.0], [658216.0, 5939298.0, 675.0], [661419.0, 5941022.0, 675.0], [663105.0, 5939454.0, 673.0], [663071.0, 5941066.0, 673.0], [656480.0, 5942480.0, 673.0], [661365.0, 5942625.0, 67

Let's create a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">line</a> resource now using this mesh.

In [27]:
boreholes = steno3d.Line(
    project=result_data,
    title='Borehole Results',
    description='Data based on results of borehole drilling',
    mesh=borehole_mesh
)
print(boreholes)

We need to add all the data to the boreholes now.

In [15]:
borehole_raw_data = Wolfpass.borehole_data
borehole_data = [
    steno3d.DataArray(title=rd, array=borehole_raw_data[rd]) for rd in borehole_raw_data
]
borehole_bound_data = [
    dict(location='CC', data=d) for d in borehole_data
]

NameError: name 'Wolfpass' is not defined

In [16]:
boreholes.data = borehole_bound_data

NameError: name 'borehole_bound_data' is not defined

These boreholes will be easier to analyze if the ground surface is also present. We can simply add the ground surface resource created for the <a href="#Inspect-the-Surface-Topography">preliminary data project</a> to our results data project.

Now our borehole results can be uploaded and shared.

In [28]:
result_data.upload()

Verifying your quota for public projects...
This PUBLIC project will be viewable by everyone.
Total progress: 100% - Uploading: project Neo4j Borehole Experiment
Complete!
https://steno3d.com/app/GMapT8wYfjyilJyHYDnZ


'GMapT8wYfjyilJyHYDnZ'

## Explore the Modeled Formation

Based on the borehole results, modelling of the dacite formation has been carried out. Let's view this modeled surface with our other results using <a href="https://steno3d.com">Steno3D</a>. We will create, upload, and print the url for sharing in one step.

In [37]:
dacite_ind = Wolfpass.lith_names.index('dacite')
dacite_vertices = Wolfpass.lith_vertices[dacite_ind]
dacite_triangles = Wolfpass.lith_triangles[dacite_ind]

dacite_surface = steno3d.Surface(
    project=result_data,
    title='Dacite Formation',
    description='Bounding surface of dacite based on borehole data and modelling results',
    mesh=steno3d.Mesh2D(
        vertices=dacite_vertices,
        triangles=dacite_triangles
    ),
    opts=dict(
        color='red'
    )
)
dacite_surface.upload()
print(result_data.url)

NameError: name 'Wolfpass' is not defined

___
**Navigation**
- <a href="#Comprehensive-Example:-Wolf-Pass-Exploration-Project">Top of page</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/seequent/steno3d-notebooks/issues/new">Report an issue</a>
___